<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Criminal-Activity-classification" data-toc-modified-id="Criminal-Activity-classification-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Criminal Activity classification</a></span><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Model-load" data-toc-modified-id="Model-load-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Model load</a></span></li><li><span><a href="#Work-directory-check" data-toc-modified-id="Work-directory-check-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Work directory check</a></span></li><li><span><a href="#Video-upload-confirmation" data-toc-modified-id="Video-upload-confirmation-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Video upload confirmation</a></span></li><li><span><a href="#Video-Classification" data-toc-modified-id="Video-Classification-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Video Classification</a></span></li><li><span><a href="#Image-upload-for-classification" data-toc-modified-id="Image-upload-for-classification-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Image upload for classification</a></span></li><li><span><a href="#Image-Classification" data-toc-modified-id="Image-Classification-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Image Classification</a></span></li><li><span><a href="#Alarm-sound-test" data-toc-modified-id="Alarm-sound-test-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Alarm sound test</a></span></li></ul></li></ul></div>

# Criminal Activity classification 

## Libraries 

In [1]:
# import the necessary packages
from keras.applications import imagenet_utils
from keras.models import load_model
import matplotlib.pyplot as plt
from collections import deque
import pandas as pd
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os

Using TensorFlow backend.
C:\Users\Student\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Student\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Student\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Student\App

## Model load 

In [2]:
# load the trained model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
model = load_model('C:/Users/Student/CrimeActivityN.model') 
lb = pickle.loads(open('C:/Users/Student/lb.Npickle', "rb").read()) 

# initialize the image mean for mean subtraction along with the
# predictions queue
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=128) #deque(maxlen=args["size"])

[INFO] loading model and label binarizer...





## Work directory check

In [5]:
os.getcwd()
# use "os.chdir("your directory path")" to change directory

'C:\\Users\\Student\\crime-activity-classification\\Project Python files'

## Video upload confirmation 

In [3]:
import cv2
import numpy as np

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture('C:/Users/Student/crime-activity-classification/Test Images and Videos/Test videos mp4/assault1.mp4')

#check if camera opened successfully
if (cap.isOpened() == False):
    print("Error opening video stream or file")
    
# Read until video is completed 
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        
        #Display the resulting frame
        cv2.imshow('Frame',frame)
        
        # Press Q on Keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    
    # Break the loop
    else:
        break
        
# When everything done, release the video capture object
cap.release()

# Close all the frames
cv2.destroyAllWindows()

## Video Classification 

In [3]:
vs = cv2.VideoCapture('C:/Users/Student/crime-activity-classification/Test Images and Videos/Test videos mp4/normal2.mp4')
writer = None
(W, H) = (None, None)

# loop over frames from the video file stream
while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()

    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
        break

    # if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]

    # clone the output frame, then convert it from BGR to RGB
    # ordering, resize the frame to a fixed 224x224, and then
    # perform mean subtraction
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame -= mean

    # make predictions on the frame and then update the predictions
    # queue
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Q.append(preds)

    # perform prediction averaging over the current history of
    # previous predictions
    results = np.array(Q).mean(axis=0)
    i = np.argmax(results)
    label = lb.classes_[i]

    # draw the activity on the output frame
    text = "Detect: {}".format(label)
    cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
        1.25, (0, 255, 0), 5)

    # check if the video writer is None
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter("C:/Users/Student/crime-activity-classification/Video Output2/normal2_128frames_smoothened2.mp4",
                                 fourcc, 30,
            (W, H), True)

    # write the output frame to disk
    writer.write(output)

    # show the output image
    cv2.imshow("Output", output)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()

[INFO] cleaning up...


## Image upload for classification 

In [142]:
# load the input image and then clone it so we can draw on it later
image = cv2.imread('C:/Users/Student/crime-activity-classification/Test Images and Videos/Test images/w.jpg')
output = image.copy()
output = imutils.resize(output, width=400)
# our model was trained on RGB ordered images but OpenCV represents
# images in BGR order, so swap the channels, and then resize to
# 224x224 (the input dimensions for VGG16)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (224, 224))
# convert the image to a floating point data type and perform mean
# subtraction
image = image.astype("float32")
mean = np.array([123.68, 116.779, 103.939][::-1], dtype="float32")
image -= mean

## Image Classification 

In [143]:
# load the trained model from disk
print("[INFO] loading model...")
# pass the image through the network to obtain our predictions
preds = model.predict(np.expand_dims(image, axis=0))[0]
i = np.argmax(preds)
label = lb.classes_[i]
# draw the prediction on the output image
text = "{}: {:.2f}%".format(label, preds[i] * 100)
cv2.putText(output, text, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
    (0, 0, 255), 2)
#if label == 'Arson' or 'Common_Assault' or 'Weapon':
 #   playsound('C:/Users/Student/Documents/My Data Science Digital Footprint/Ominous Beeping App.mp3')

# show the output image
cv2.imshow("Output", output)
cv2.waitKey(0)

[INFO] loading model...


-1

In [144]:
n = [0,1,2,3]

In [145]:
for value in n:
    print("{}: {:.2f}%".format(lb.classes_[value], preds[value] * 100))

Arson: 0.05%
Common_Assault: 6.67%
Weapon: 93.27%


## Alarm sound test

In [ ]:
from playsound import playsound

In [ ]:
playsound('C:/Users/Student/Documents/My Data Science Digital Footprint/Ominous Beeping App.mp3')